In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


import time

# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC




from datetime import datetime
import pandas as pd

In [2]:
urls = {
    'avito.ru' : "https://www.avito.ru/rossiya/avtomobili?cd=1&radius=0&q=",
    'auto.ru' : "https://auto.ru/cars/",         # + /all
    'drom.ru' : "https://auto.drom.ru"         
}

In [3]:
class Parsers:
    def __init__(self, urls, driver : Chrome) -> None:
        self.urls = urls
        self.driver = driver

    def get_avito(self, search_query : str, reopened : bool = False, datas : pd.DataFrame = pd.DataFrame()):
        self.cars_datas = datas

        if not reopened:
            self.driver.get(self.urls['avito.ru'] + search_query)

        # cars_panels = self.driver.find_elements(By.CSS_SELECTOR, 'div.iva-item-root-_lk9K.photo-slider-slider-S15A_.iva-item-list-rfgcH.iva-item-redesign-rop6P.iva-item-responsive-_lbhG.iva-item-xl-_jicv.items-item-My3ih.items-listItem-Gd1jN.js-catalog-item-enum')
        cars_panels = self.driver.find_elements(By.XPATH, '//*[@data-marker="item"]')


        car_data = {}
        # print(len(cars_panels))


        for car_panel in cars_panels:
            publication_name = car_panel.find_element(By.CLASS_NAME, 'iva-item-titleStep-pdebR').text

            # print(publication_name.split(','))

            publication_name = publication_name.split(',')

            if len(publication_name) == 3:
                name, year, mileage = publication_name
            
            elif len(publication_name) == 4:
                name, year, _, mileage = publication_name




            car_data.update({
                'car_model' : name,
                'car_year_produced' : year,
                'car_mileage' : int(mileage.replace(' ', '')[: -3]),
                'car_price' : ''.join(car_panel.find_element(By.CLASS_NAME, 'price-root-RA1pj').text.split(' '))[: -1],
                'publication_link' : car_panel.find_element(By.CLASS_NAME, 'iva-item-title-py3i_').find_element(By.CSS_SELECTOR, 'a.styles-module-root-QmppR.styles-module-root_noVisited-aFA10').get_attribute('href')

            })

            self.cars_datas = pd.concat([self.cars_datas, pd.DataFrame([car_data])])


            # print(car_data)





        # pages_panel = self.driver.find_element(By.CSS_SELECTOR, 'ul.styles-module-root-nZne3.styles-module-root_size_s-FzmdD')

        try:
            # next_button = self.driver.find_element(By.CSS_SELECTOR, 'a.styles-module-item-kF45w.styles-module-item_arrow-sxBqe.styles-module-item_size_s-Tvz95.styles-module-item_link-_bV2N')
            next_button = self.driver.find_element(By.XPATH, '//*[@data-marker="pagination-button/nextPage"]')

            # print(next_button.text)
            
            # return self.cars_datas
            # break
            next_button.click()
            self.get_avito(search_query=search_query, reopened=True, datas=self.cars_datas)

        except:
            # Последняя страница
            return(self.cars_datas.reset_index())


    def get_drom(self, mark : str, model : str, year : str, reopened : bool = False, datas : pd.DataFrame = pd.DataFrame()):
        self.cars_datas = datas

        if not reopened:
            self.driver.get(self.urls['drom.ru'])
            self.driver.refresh()

        mark_input = self.driver.find_element(By.XPATH, '//input[@placeholder="Марка"]')
        model_input = self.driver.find_element(By.XPATH, '//input[@placeholder="Модель"]')   

        variants = self.driver.find_elements(By.CSS_SELECTOR, 'div.css-1vc4kf7.ewaf0l10')
        mark_variants = variants[0]
        model_variants = variants[1]


        mark_input.click()
        mark_input.send_keys(mark)
        time.sleep(1)
        mark_variants.find_elements(By.CSS_SELECTOR, 'div.css-1r0zrug.e1uu17r80')[2].click()


        model_input.click()
        model_input.send_keys(model)
        time.sleep(1)
        model_variants.find_elements(By.CSS_SELECTOR, 'div.css-1r0zrug.e1uu17r80')[2].click()


        year_button = self.driver.find_elements(By.CSS_SELECTOR, 'button.css-me79aa.e75dypj1')[1]
        year_button.click()

        year_input= self.driver.find_element(By.CSS_SELECTOR, 'div.css-u25ii9.e154wmfa0')
        years = year_input.find_elements(By.CSS_SELECTOR, 'div.css-10ya1wz.e1x0dvi10')

        year = [year_ for year_ in years if year_.text == year][0]

        year.click()

        self.driver.find_element(By.CSS_SELECTOR, 'button.css-1txnc2i.e3cb8x01').click()

        







    def test_avito_next(self):
        self.driver.get(urls['avito.ru'])
        next_button = self.driver.find_element(By.XPATH, '//*[@data-marker="pagination-button/nextPage"]')

        # print(next_button.text)
        
        # return self.cars_datas
        # break
        next_button.click()
        # self.get_avito(search_query=search_query)







---

In [4]:
parsers = Parsers(urls, Chrome())

---

In [8]:
avito_cars = parsers.get_avito('BMW x6 2020')
print(avito_cars)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.185)
Stacktrace:
	GetHandleVerifier [0x00007FF7ACD45E42+3538674]
	(No symbol) [0x00007FF7AC964C02]
	(No symbol) [0x00007FF7AC815AEB]
	(No symbol) [0x00007FF7AC7F288C]
	(No symbol) [0x00007FF7AC885DD7]
	(No symbol) [0x00007FF7AC89B40F]
	(No symbol) [0x00007FF7AC87EE53]
	(No symbol) [0x00007FF7AC84F514]
	(No symbol) [0x00007FF7AC850631]
	GetHandleVerifier [0x00007FF7ACD76CAD+3738973]
	GetHandleVerifier [0x00007FF7ACDCC506+4089270]
	GetHandleVerifier [0x00007FF7ACDC4823+4057299]
	GetHandleVerifier [0x00007FF7ACA95C49+720121]
	(No symbol) [0x00007FF7AC97126F]
	(No symbol) [0x00007FF7AC96C304]
	(No symbol) [0x00007FF7AC96C432]
	(No symbol) [0x00007FF7AC95BD04]
	BaseThreadInitThunk [0x00007FF97ED77344+20]
	RtlUserThreadStart [0x00007FF97F7C26B1+33]


---

In [5]:
drom_cars = parsers.get_drom('BMW', 'X6', '2020')

In [6]:
mark_input = parsers.driver.find_element(By.XPATH, '//input[@placeholder="Марка"]')
model_input = parsers.driver.find_element(By.XPATH, '//input[@placeholder="Модель"]')

In [9]:
[element.text for element in parsers.driver.find_elements(By.CSS_SELECTOR, 'div.css-1r0zrug.e1uu17r80')]

['Любая марка', '', 'BMW (15824)', '', '']

In [32]:
mark_input.click()
mark_input.send_keys('BMW')
# mark_input.send_keys('')
# mark_input.send_keys('AAAAAA')

In [9]:
parsers.driver.find_elements(By.CSS_SELECTOR, 'div.css-1r0zrug.e1uu17r80')[2].click()

In [6]:
parsers.driver.find_elements(By.CSS_SELECTOR, 'div.css-1vc4kf7.ewaf0l10')

[<selenium.webdriver.remote.webelement.WebElement (session="f2a0cfaca02c1de12294eb1209cfade5", element="8B43CEE97E9BF8E9AE0E25680F9BE29B_element_650")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f2a0cfaca02c1de12294eb1209cfade5", element="8B43CEE97E9BF8E9AE0E25680F9BE29B_element_665")>]

In [19]:
mark_input.clear()


In [ ]:
parsers.driver.send